Extract specimen candidates from the references of the publications to use in training a model. This necessitated a different step because of the way the references were nested in json.
I used reg ex pattern to match string values made up of 3- 6 letters follewed by several numbers, such as USNM 2334521. Then I saved the sentence containing the specimen in a csv to be used in the next step.

In [1]:
import pandas as pd
import numpy as np
import re
import glob
import random
from random import sample
import json
import jsonlines
import pysbd
seg = pysbd.Segmenter(language='en', clean=False)
import spacy
from spacy import displacy

In [2]:
all_papers = glob.glob('./papers_train/*')

In [3]:
print(f"Looking for SPECIMEN entities in {len(all_papers)} TEST papers")

Looking for SPECIMEN entities in 400 TEST papers


In [4]:
for name in all_papers:
    print(name)

./papers_train\002b2f92-8dc5-7bd9-6689-ef79f8c3c461.json
./papers_train\002f9cc4-096b-faff-f5b7-751f497e28aa.json
./papers_train\00513063-020a-ca12-055d-fed45fa7cb00.json
./papers_train\00c99f27-6e4d-d42f-3306-f416b09b8163.json
./papers_train\00f24603-8295-1fe0-52c5-53b5d2f74429.json
./papers_train\01847803-d840-fccb-2446-eceecaa0a679.json
./papers_train\01e50dde-46ca-2ca5-90da-4776dbf9c505.json
./papers_train\02352cfc-43d3-c1cd-4443-6c48cfbb5634.json
./papers_train\0262a5aa-edec-31ef-5762-762620379089.json
./papers_train\0361524c-2cc0-f859-aa65-24ab713b7e05.json
./papers_train\047cd56e-fd6c-c364-8191-cbeb175bf448.json
./papers_train\0584d26c-f724-3a0c-0947-bf82b8ed3aa4.json
./papers_train\0615594c-3ae1-ab40-0dfe-24b90d1b85c9.json
./papers_train\065a13d4-32e9-933d-649e-0f3a63e766b7.json
./papers_train\06714938-04fe-973d-261c-ec19fff83cec.json
./papers_train\067fece7-f1ff-06dc-f45a-a79bd855fb35.json
./papers_train\07991818-0e7b-dcac-cf1d-04cc8eb0f052.json
./papers_train\07b52a8b-0af0-1e

RefEntriesParse stores, in a list called reflist, each tag that begins with "FIGREF or "TABREF" (such as "FigRef0, Figref1, FigRef2" Json2text then iterates through reglist and returns all the text from the references. 

In [5]:
def RefEntriesParse(file):
    with open(file, 'r') as f:
        reflist = []    
        data = json.loads(f.read()) ## dictionary
        for k,v in data["pdf_parse"]["ref_entries"].items():
            if k.startswith("FIGREF") or k.startswith("TABREF"):
                ref_label = k
                reflist.append(ref_label)
    return(reflist)

In [6]:
def json2text(file, refID):
    with open(file, 'r') as f:
        data = json.loads(f.read())
        ref_text_df = pd.json_normalize(data["pdf_parse"]["ref_entries"][refID])
    return(ref_text_df) 

In [7]:
def text2sentences(ref_text_df):
    sentences_text = " ".join(list(ref_text_df.text)) #sentences_text is a string
    return(sentences_text)

In [8]:
def sentences2sentencelist(text):
    sentences = seg.segment(text) #sentences is a list of strings
    sentences = [re.sub(r"^\W+", "", sentence) for sentence in sentences] 
    sentences = [re.sub(r"\s+", " ", sentence) for sentence in sentences]
    return(sentences)

In [9]:
mylist =  []
for file in all_papers:
    reflist = RefEntriesParse(file)
    for refID in reflist:
        try:
            ref_text_df = json2text(file, refID)
            sentences_text = text2sentences(ref_text_df)
            sentencelist = sentences2sentencelist(sentences_text)
            mylist.extend(sentencelist)
        except:
            print(file + " returns error.")
            continue

In [10]:
print(len(mylist))

21795


In [11]:
print(mylist)

['Cytosine methylation in DNA. ', 'A) Addition of a methyl group, CH 3 (red), at the five position of the cytosine pyrimidine ring (black arrow) does not sterically interfere with GC base pairing (blue lines). ', 'DNA methyltransferases associate covalently with the carbon 6 position (straight green arrow) during methyl group transfer. ', 'B) A model of B-form DNA methylated at cytosines in two self-complementary CpG sequences. ', 'The paired methyl moieties (magenta and yellow) lie in the major groove of the double helix.DNA Methylation in Mammals Cite this article as Cold Spring Harb Perspect Biol 2014;6:a019133', 'et', 'Methylation in Mammals Cite this article as Cold Spring Harb Perspect Biol 2014;6:a019133', 'Methylation in Mammals Cite this article as Cold Spring Harb Perspect Biol 2014;6:a019133', 'A A A A A A A Ac c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c c', 'Function of mammalian DNA methyltransferases', 'CpG islands. ',

In [12]:
pattern = r"(([A-Z]{3,6})[_\-\s]?\w*[_\-\s]?[0-9]{3,10})"
candidates = []
for s in mylist:
    matches = re.findall(pattern, s)
    if len(matches) > 0:
        sent = s
        match = matches
        list_item = []
        list_item = [[sent, match]]
        candidates.extend(list_item)

In [13]:
print(candidates)

[['T y p e , h o s t a n d locality d a t a -Holotype female and 6 paratype females from the occiput area near the tentorial pits of a female Apis mellifera (Hymenoptera: Apidae), intercepted with honey bee comb (1 kg) from Mexico at the port of entry (PPQ-309 # 037169, SEL # 0200689) in Brownsville, Texas, coll. ', [('PPQ-309', 'PPQ')]], ['Oral valves and palatal groove in dorsal aspect of oral ca vity-(A) Toxotes c hatareus (KUI 42697), arrow I points to left dorsal oral valve, arrow II points to palatal groove, ventral view, scale bar 1 mm. ', [('KUI 42697', 'KUI')]], ['Groove in ventral aspect of parasphenoid (character state 5 1 )-(B) T. jaculatrix (FMNH 69510), arrow points to groove in parasphenoid, ventral view, scale bar 1 mm. ', [('FMNH 69510', 'FMNH')]], ['Basihyal inserts above basibranchial one and covers the first basibranchial when viewing the branchial and hyoid arches dorsally (character state 50 1 )-(C) T. c hatareus (KUI 42697), ar row points to basibranchial three, 

In [14]:
print(candidates[1])

['Oral valves and palatal groove in dorsal aspect of oral ca vity-(A) Toxotes c hatareus (KUI 42697), arrow I points to left dorsal oral valve, arrow II points to palatal groove, ventral view, scale bar 1 mm. ', [('KUI 42697', 'KUI')]]


In [15]:
candidates_df = pd.DataFrame(candidates)
print(candidates_df)

                                                     0  \
0    T y p e , h o s t a n d locality d a t a -Holo...   
1    Oral valves and palatal groove in dorsal aspec...   
2    Groove in ventral aspect of parasphenoid (char...   
3    Basihyal inserts above basibranchial one and c...   
4    For all images of an isolated basihyal, leftmo...   
..                                                 ...   
926  This content downloaded from 35.7.38.24 on Wed...   
927  This content downloaded from 35.7.38.24 on Wed...   
928  Four eastern deciduous forest species occur oc...   
929  This content downloaded from 35.7.38.24 on Wed...   
930  This content downloaded from 35.7.38.24 on Wed...   

                                                     1  
0                                     [(PPQ-309, PPQ)]  
1                                   [(KUI 42697, KUI)]  
2                                 [(FMNH 69510, FMNH)]  
3              [(KUI 42697, KUI), (USNM 454834, USNM)]  
4    [(USNM 454833

In [16]:
candidates_df.to_csv("Candidates_Ref.csv", index=False, header=False)